# 1. SHP파일 불러오기

In [ ]:
# 라이브러리 임포트
import os
from dotenv import load_dotenv
import geopandas as gpd
from pathlib import Path
from sqlalchemy import create_engine, text
# pip install geoalchemy2    # to_postgis를 쓸 때 필요한 패키지: PostGIS 테이블로 바로 저장해 주는 함수

In [ ]:
# .env 파일 로드
load_dotenv()

# DB 정보
host = os.getenv("DB_HOST")
database = os.getenv("DB_NAME")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")
port = int(os.getenv("DB_PORT", 5432))
client_encoding = "UTF8"

In [5]:
# shp 파일 경로
path = r"D:\gis_test\AL_D002_36_20260104.shp".replace('\\','\\\\')
# geopandas
gdf = gpd.read_file(path)

In [34]:
# 컬럼명 변경
gdf = gdf.rename(columns={
    "A0": "shape_id",
    "A1": "gis_building_uid",
    "A2": "unique_id",
    "A3": "legal_dong_code",
    "A4": "legal_dong_name",
    "A5": "special_zone_code",
    "A6": "special_zone_name",
    "A7": "lot_number",
})

In [38]:
print("geometry type:",geom.geom_type)    
print("좌표계:",gdf.crs)
display(gdf.head())

geometry type: Polygon
좌표계: EPSG:5186


,shape_id,gis_building_uid,unique_id,legal_dong_code,legal_dong_name,special_zone_code,special_zone_name,lot_number,geometry
0,468421,3611010100200550000,3611010100,세종특별자치시 반곡동,55,산55임,2026-01-01,36110,"POLYGON ((227473.094 432595.853, 227475.965 43..."
1,1395253,3611010100102030335,3611010100,세종특별자치시 반곡동,203-335,203-335 답,2026-01-01,36110,"POLYGON ((228112.406 431647.352, 228114.599 43..."
2,1434968,3611010100111530000,3611010100,세종특별자치시 반곡동,1153,1153 도,2026-01-01,36110,"POLYGON ((228153.197 431657.597, 228153.469 43..."
3,1417981,3611010100102030111,3611010100,세종특별자치시 반곡동,203-111,203-111장,2026-01-01,36110,"POLYGON ((228313.015 431679.854, 228307.984 43..."
4,1434967,3611010100111540000,3611010100,세종특별자치시 반곡동,1154,1154 도,2026-01-01,36110,"POLYGON ((228254.947 431686.278, 228255.018 43..."


# 2. SQL Alchemy 엔진 생성

In [40]:
# 연결 문자열 (psycopg2 드라이버 사용)
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/cadastre")

# 연결 테스트
with engine.connect() as conn:
    result = conn.execute(text("SELECT version();"))
    print(result.fetchone())

('PostgreSQL 17.4 on x86_64-windows, compiled by msvc-19.42.34436, 64-bit',)


In [ ]:
# 테이블 생성 (DDL)
create_sql = """
CREATE TABLE IF NOT EXISTS cadastral_polygon (
    gis_building_uid VARCHAR,
    shape_id VARCHAR,
    unique_id VARCHAR,
    legal_dong_code VARCHAR,
    legal_dong_name VARCHAR,
    special_zone_code VARCHAR,
    special_zone_name VARCHAR,
    lot_number VARCHAR,
    geometry GEOMETRY(MULTIPOLYGON, 5186)
);
"""

with engine.connect() as conn:
    conn.execute(text(create_sql))
    conn.commit()

#### pip install geoalchemy2
to_postgis를 쓸 때 필요한 패키지

In [47]:
gdf.to_postgis(
    name="cadastral_polygon",
    con=engine,
    schema="public",
    if_exists="append",   # 기존 테이블에 추가
    index=False
)

## 전체 shp 업로드

In [97]:
# 연결 문자열 (psycopg2 드라이버 사용)
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/cadastre")

# 저체 shp 경로 불러기기
folder = Path(r"D:\gis_test")
shp_paths = list(folder.glob("*.shp"))

In [98]:
# 파일 번호별 지역명
region_name ={
    "11": "seoul",
    "26": "busan",
    "27": "daegu",
    "28": "incheon",
    "29": "gwangju",
    "30": "daejeon",
    "31": "ulsan",
    "36": "sejong",
    "41": "gyeonggi",
    "43": "chungcheongbuk",
    "44": "chungcheongnam",
    "46": "jeollanam",
    "47": "gyeongsangbuk",
    "48": "gyeongsangnam",
    "50": "jeju",
    "51": "gangwon",
    "52": "jeonbuk",
}

In [102]:
def create_table(name):
    create_sql = f"""
    CREATE TABLE IF NOT EXISTS {name} (
        gis_building_uid VARCHAR,
        shape_id VARCHAR,
        unique_id VARCHAR,
        legal_dong_code VARCHAR,
        legal_dong_name VARCHAR,
        special_zone_code VARCHAR,
        special_zone_name VARCHAR,
        lot_number VARCHAR,
        geometry GEOMETRY(MULTIPOLYGON, 5186)
    );
    """
    
    with engine.connect() as conn:
        conn.execute(text(create_sql))
        conn.commit()

In [100]:
for path in shp_paths:
    number = path.name.split("AL_D002_")[1][:2]
    name = region_name[number]
    print(name)
    create_table(name)
    
    # shp파일 읽기
    gdf = gpd.read_file(path)
    # 컬럼명 변경
    gdf = gdf.rename(columns={
        "A0": "shape_id",
        "A1": "gis_building_uid",
        "A2": "unique_id",
        "A3": "legal_dong_code",
        "A4": "legal_dong_name",
        "A5": "special_zone_code",
        "A6": "special_zone_name",
        "A7": "lot_number",
    })
    # PostgreSQL public 스키마 cadastral_polygon 테이블 저장
    gdf.to_postgis(
        name=name,
        con=engine,
        schema="public",
        if_exists="append",   # 기존 테이블에 추가
        index=False
    )

Seoul
Busan
Daegu
Incheon
Gwangju
Daejeon
Ulsan
Sejong
Gyeonggi
Gyeonggi
Gyeonggi
Gyeonggi
Gyeonggi
Gyeonggi
Chungcheongbuk
Chungcheongbuk
Chungcheongbuk
Chungcheongnam
Chungcheongnam
Chungcheongnam
Chungcheongnam
Jeollanam
Jeollanam
Jeollanam
Jeollanam
Jeollanam
Jeollanam
Gyeongsangbuk
Gyeongsangbuk
Gyeongsangbuk
Gyeongsangbuk
Gyeongsangbuk
Gyeongsangbuk
Gyeongsangnam
Gyeongsangnam
Gyeongsangnam
Gyeongsangnam
Gyeongsangnam
Jeju


D:\anaconda3\Lib\site-packages\pyogrio\raw.py:200: RuntimeWarning: D:\gis_test\AL_D002_50_20260104.shp contains polygon(s) with rings with invalid winding order. Autocorrecting them, but that shapefile should be corrected using ogr2ogr for example.
  return ogr_read(


Gangwon
Gangwon
Gangwon
Jeonbuk
Jeonbuk
Jeonbuk
Jeonbuk
